We need to collect headline data for stocks. We're going to start with one stock, google class C ($GOOG). And we also need a variety of sources. The more sources the better. We can always trim the fat.

yfinance will give us a good dataset for Yahoo finance data.

Since we're using Python, we need to rely on websites that are builf in HTML to not make this too over;y complicated. If most of our data sources are buiit in React or are JS- heavy, we'll switch to using JS.

News Sources:
Yahoo Finance
CNBC
BizToc
Reuters


In [ ]:
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [86]:
url = "https://biztoc.com/wire"

response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, "html.parser")

all_links = [a['href'] for a in soup.find_all('a', href=True)]
print(f"Found {len(all_links)} links:")

valid_urls = list(url for url in all_links if url and url.startswith(('http://', 'https://')))
valid_urls = valid_urls[6:]

Found 258 links:


In [87]:


def get_valid_links(url):
    try:
        response = requests.get(url, timeout=10, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        links = set()  # Avoid duplicates
        for a in soup.find_all('a', href=True):
            href = a['href'].strip()
            if (href and not href.startswith(('javascript:', '#', 'mailto:', 'tel:'))
                and not any(href.endswith(ext) for ext in ('.png', '.jpg', '.pdf', '.docx'))):
                
                absolute_url = urljoin(url, href)
                if urlparse(absolute_url).scheme in ('http', 'https'):  # Validate URL
                    links.add(absolute_url)
        return list(links)
    
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return []

def get_headline_and_text(url):
    service = Service()  # Specify your path
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(url)
        # Wait for main content to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body')))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Better title extraction (fallback to URL)
        title = soup.title.string if soup.title else url
        
        # Cleaner text extraction (skip nav/footer)
        main_content = soup.find('main') or soup.find('article') or soup.find('body')
        text = ' '.join([p.get_text(' ', strip=True) 
                        for p in main_content.find_all(['p', 'h1', 'h2', 'h3']) 
                        if p.get_text(strip=True)])
        
        return title, text
    
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None, None
    
    finally:
        driver.quit()  # Ensure driver closes





In [89]:
import time
import random
import pandas as pd
from tqdm import tqdm  # Progress bar
from datetime import datetime

# Configuration
MAX_LINKS = 10  # Prevent runaway scraping
output_data = []

# Scrape with progress tracking
for link in tqdm(valid_urls[:MAX_LINKS]):
    try:
        title, text = get_headline_and_text(link)
        if title and text:  # Only store valid data
            output_data.append({
                'url': link,
                'headline': title,
                'body': text,
                'date': datetime.now().strftime('%Y-%m-%d')  # Placeholder
            })
        time.sleep(random.uniform(1, 5))  # Be polite
    
    except Exception as e:
        print(f"Skipping {link} due to error: {e}")
        continue

# Convert to DataFrame once
df = pd.DataFrame(output_data)

 10%|█         | 1/10 [00:05<00:46,  5.18s/it]

Error scraping https://theweek.com/culture-life/americas-favorite-fast-food-restaurants?ref=biztoc.com: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.55)
Stacktrace:
0   chromedriver                        0x0000000105314708 cxxbridge1$str$ptr + 2729312
1   chromedriver                        0x000000010530c96c cxxbridge1$str$ptr + 2697156
2   chromedriver                        0x0000000104e5e728 cxxbridge1$string$len + 90444
3   chromedriver                        0x0000000104e38744 chromedriver + 132932
4   chromedriver                        0x0000000104ecdc9c cxxbridge1$string$len + 546496
5   chromedriver                        0x0000000104ee6be0 cxxbridge1$string$len + 648708
6   chromedriver                        0x0000000104e99bc0 cxxbridge1$string$len + 333284
7   chromedriver                        0x00000001052d8298 cxxbridge1$str$ptr + 2482416
8   chromedriver                        0x000000

 20%|██        | 2/10 [00:26<01:57, 14.74s/it]

Error scraping https://thehill.com/homenews/house/5320723-congressional-republicans-tax-spending-cut/?ref=biztoc.com: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.55)
Stacktrace:
0   chromedriver                        0x0000000104ea8708 cxxbridge1$str$ptr + 2729312
1   chromedriver                        0x0000000104ea096c cxxbridge1$str$ptr + 2697156
2   chromedriver                        0x00000001049f2728 cxxbridge1$string$len + 90444
3   chromedriver                        0x00000001049cc744 chromedriver + 132932
4   chromedriver                        0x0000000104a61c9c cxxbridge1$string$len + 546496
5   chromedriver                        0x0000000104a7abe0 cxxbridge1$string$len + 648708
6   chromedriver                        0x0000000104a2dbc0 cxxbridge1$string$len + 333284
7   chromedriver                        0x0000000104e6c298 cxxbridge1$str$ptr + 2482416
8   chromedriver                  

 60%|██████    | 6/10 [01:10<00:39,  9.93s/it]

Error scraping https://www.fiercebiotech.com/biotech/eli-lilly-lands-next-gen-pain-asset-siteone-therapeutics-acquisition-worth-1b?ref=biztoc.com: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.55)
Stacktrace:
0   chromedriver                        0x0000000102ccc708 cxxbridge1$str$ptr + 2729312
1   chromedriver                        0x0000000102cc496c cxxbridge1$str$ptr + 2697156
2   chromedriver                        0x0000000102816728 cxxbridge1$string$len + 90444
3   chromedriver                        0x00000001027f0744 chromedriver + 132932
4   chromedriver                        0x0000000102885c9c cxxbridge1$string$len + 546496
5   chromedriver                        0x000000010289ebe0 cxxbridge1$string$len + 648708
6   chromedriver                        0x0000000102851bc0 cxxbridge1$string$len + 333284
7   chromedriver                        0x0000000102c90298 cxxbridge1$str$ptr + 2482416
8   c

100%|██████████| 10/10 [01:53<00:00, 11.33s/it]


In [90]:
df

,url,headline,body,date
0,https://www.zerohedge.com/medical/rogan-rodger...,Rogan & Rodgers Unleash Big Pharma's Worst Nig...,Rogan & Rodgers Unleash Big Pharma's Worst Nig...,2025-05-27
1,https://www.nytimes.com/2025/05/27/us/politics...,Republicans Urge Trump to Put New Sanctions on...,Russia-Ukraine War Advertisement Supported by ...,2025-05-27
2,https://www.benzinga.com/insights/news/25/05/4...,$1000 Invested In Marsh & McLennan Cos 10 Year...,$1000 Invested In Marsh & McLennan Cos 10 Year...,2025-05-27
3,https://www.nytimes.com/2025/05/27/us/politics...,Judge Strikes Down Trump Order Targeting Wilme...,Trump Administration Advertisement Supported b...,2025-05-27
4,https://www.cnbc.com/2025/05/27/circle-ipo-has...,Circle IPO has peculiar Facebook-like characte...,Circle IPO has peculiar Facebook-like characte...,2025-05-27
5,https://www.theglobeandmail.com/business/comme...,A Throne Speech fit for a king. But where’s th...,A Throne Speech fit for a king. But where’s th...,2025-05-27
6,https://thehill.com/business/5320681-tesla-sha...,Tesla stock recovers as Musk shifts focus back...,Tesla shares rise after Musk says he’s back at...,2025-05-27
